In [1]:
from sudoku import Board

In [30]:
B = Board(9)

In [31]:
B.solve()

True

In [32]:
B.prepare(2)

In [33]:
B.solution()

array([[9, 3, 6, 7, 8, 5, 2, 4, 1],
       [8, 7, 4, 6, 2, 1, 3, 9, 5],
       [5, 1, 2, 9, 3, 4, 8, 6, 7],
       [6, 5, 1, 4, 9, 2, 7, 8, 3],
       [4, 2, 8, 3, 5, 7, 9, 1, 6],
       [3, 9, 7, 8, 1, 6, 4, 5, 2],
       [1, 8, 9, 2, 6, 3, 5, 7, 4],
       [2, 4, 5, 1, 7, 8, 6, 3, 9],
       [7, 6, 3, 5, 4, 9, 1, 2, 8]])

In [71]:
step_count = 9
height = 270
width = 270
image = Image.new(mode='L', size=(height, width), color=255)
font = ImageFont.truetype('Hack.ttf', size=25)
# Draw some lines
draw = ImageDraw.Draw(image)
y_start = 0
y_end = image.height
step_size = int(image.width / step_count)

for x in range(0, image.width, step_size):
    line = ((x, y_start), (x, y_end))
    draw.line(line, fill=0)

x_start = 0
x_end = image.width

for y in range(0, image.height, step_size):
    line = ((x_start, y), (x_end, y))
    draw.line(line, fill=0)

    
for x in range(0, image.width, step_size*3):
    line = ((x-1, y_start), (x-1, y_end))
    draw.line(line, fill=0)
    line = ((x+1, y_start), (x+1, y_end))
    draw.line(line, fill=0)

for y in range(0, image.height, step_size*3):
    line = ((x_start, y-1), (x_end, y-1))
    draw.line(line, fill=0)
    line = ((x_start, y+1), (x_end, y+1))
    draw.line(line, fill=0)
draw.text((8,0),'5',font=font)
del draw

image.save('drawn_grid.png')

In [51]:
from PIL import Image, ImageFont, ImageDraw
im = Image.open("drawn_grid.png")
helvetica = ImageFont.truetype(filename="Helvetica.ttf", size=40)
d = ImageDraw.Draw(im)

location = (100, 50)
text_color = (100, 100, 200)
d.text(location, "1", font=helvetica, fill=text_color)

im.save("drawn_grid.png")

TypeError: truetype() got an unexpected keyword argument 'filename'

In [72]:
for i in B.grid:
    print(i)

[0 0 0 0 8 0 2 0 0]
[0 0 0 0 0 1 3 9 0]
[0 0 2 0 3 4 0 0 0]
[0 0 1 4 0 2 0 0 0]
[4 2 0 0 0 0 0 1 6]
[3 9 7 0 1 0 4 5 0]
[1 8 9 2 0 3 0 7 0]
[2 0 5 1 0 0 6 0 0]
[0 6 0 0 4 9 0 0 0]
